##Procedure to get point sources and assemble observing lists for Speckle Imaging using scripts and instructions from Mark Everett. Currently all scripts run in Python 2. May require Chrome browser.

###Example files to run scripts below can be found in `/content/gdrive/My Drive/Obs_Prep_Notebook/Example_files`

##1) Mount Google Drive and set up Python.

In [0]:
# Mount google drive:

from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Set up Python:

import os
import re
import sys
sys.path.append('/content/gdrive/My Drive/Obs_Prep_Notebook')

import myutils
from targetlist_utils import makeOlistFile1, filechecker

##2) Upload text file with object names, coordinates, magnitude, and optional proper motions (*.coords*).

The coordinates should be hexagesimal, RA should be in hours of RA and proper motions in milliarcsec/year. A positive sign for declination is optional. The coordinates can have whitespace or colon separators, but only one style or the other should be used on a given line.  Normally the input file coordinates should be J2000. Separators between columns is whitespace. Proper motions are optional. To associate a PI and optionally a subgroup/Co-I for each object in the coords file use 'header' lines with the format "#PI:" or "#PI_COI:". Blank lines are ignored and text following a "#" can be ignored except when it is in exactly the format above.

Example coordinate file format (also see Howell.coords in Speckle Group Google Drive)

```
#HOWELL:
NAME     HH:MM:DD.DDD +DD:MM:SS.SSS MAG PMRA PMDec
NAME     HH MM DD.DDD +DD MM SS.SSS MAG
#HOWELL_CALIB:
LHS1140  00:44:59.3 -15:16:17   14.1    317    -589
HD3167   00:34:57.5 +04:22:53    8.9    107.58 -173.14 #priority 1

```



In [0]:
# Upload coords file from local computer:

from google.colab import files
uploaded = files.upload()

Saving Howell.coords to Howell.coords


##3) Create target list with PSF calibrators in 'our' format (*.wlist*) or for submission to Gemini (*.gemini*)

*  This script (copied from `make_wlist.py`) will ask you for which **telescope** you will be observing at (WIYN, GN, GS) and which **format** you want the output file to be in (our list vs. Gemini format). Our format is useful for creating observing plans for use at the telescope, while the Gemini format is a CSV-formatted file that Gemini can injest into its Phase 2 program as a bulk upload.

*  You will then be asked for the **input data file** (*coords* file), if you want to **add point sources** for each target (typically yes), and whether you want to **include trailing comments** (notes listed after # in coords file). Trailing comments can be useful for notes at the telescope if the output file (*wlist*) is used to create an olist, or observing plan.

*  The next thing it will ask you is how you want **H-numbers** to be assigned to targets. We usually select 's' and input a 6-digit starting number (typically 600000). It will also ask you to do this when creating a Gemini formatted file, but the H number won't be used. Finally it asks for the name of a *wlist* file to create, which is the **output file**.

*  Note that the order of the input list corresponds to the output order, meaning that even if you change the output format (*wlist* vs. *gemini*), the output file should correspond line-for-line to a run of the same script using the other output format.

In [0]:
# Run make_wlist.py:

maxnumps = 1 #the maximum number of point sources chosen per science target
searchrad = 5 #degrees radius around science target to search for point sources
telescopes = ('WIYN','GN','GS') #default will be first telescope listed
telescope=myutils.ask_user(
    'Pick telescope ({} or {})? [{}]'.format(
        ", ".join(telescopes[0:-1]),
        telescopes[-1],
        telescopes[0]),
    default=telescopes[0],allowed=telescopes
)
if telescope=='GN':
    telescopeDEC=19.8238
elif telescope=='GS':
    telescopeDEC=-30.2406
elif telescope=='WIYN':
    telescopeDEC=31.958
formt = myutils.ask_user("Pick 'w' for wlist or 'g' for Gemini format [w]:", default='w', allowed=('w','g'))
if formt == 'w':
    ourformat = True
else:
    ourformat = False

#print '-----------------------------------------------------------------'
#print "Input files have lines denoting target ownership, starting with '#'"
#print "and ending with ':'.  Other comment lines start with '#' and"
#print "text following '#' in any other line is treated as a comment."
#print "Target data lines have 4, 6, 8 or 10 whitespace-separated columns."
#print "The data entries are, in order, NAME, RA, DEC, mag, pmRA, pmDEC."
#print "The proper motion data are optional."
#print "RA and DEC are hexagesimal and can be either whitespace or :-delimited."
#print "Proper motions are in arcseconds/year."
#print "The Equinox and epoch should both be 2000.0"
#print 'E.g.,:'
#print '#HOWELL:'
#print 'HD217107     22:58:15.54 -02:23:43.25  6.160 -6.000 -16.000'
#print 'EP212575828    13 40 38.328    -11 07 01.53    15.508 #high priority'
print '-----------------------------------------------------------------'
print 'make_wlist is running with these settings:'
print 'Max number of point sources/science target =',maxnumps
print 'Format =',
if ourformat:
    print 'wlist format'
else:
    print 'Gemini target submission format'
print 'Telescope =',telescope,'at Latitude =',telescopeDEC
print '-----------------------------------------------------------------'

    
inputfile = ''
while not os.path.exists(inputfile):
    if len(sys.argv)>1:
        query = 'Name of input data file [{}]:'.format(sys.argv[1])
        inputfile = myutils.ask_user(query, default=sys.argv[1])    
    else:
        query = 'Name of input data file :'
        inputfile = myutils.ask_user(query)    
    if not os.path.exists(inputfile):
        print '%s does not exist' % inputfile

if filechecker(inputfile)!=0:
    exit()

if telescope in ('WIYN',):            
    pmyears = float(myutils.ask_user('Years of proper motion to add to input coords: [0.]',default='0.'))
else:
    pmyears = 0.0

psources = myutils.ask_user('Do you want a point source added for each target (y/n)? [y]',
                            default='y',allowed=('y','n'))
if psources == 'y':
    includePointSources = True
else:
    includePointSources = False

comments =  myutils.ask_user('Include trailing comments in the output file (y/n)? [n]',
                            default='n',allowed=('y','n'))
if comments == 'y':
    includeComments = True
else:
    includeComments = False

startnumber=0
numbering=None
if ourformat==True:
    naming = myutils.ask_user(
        'How should H-numbers be assigned to targets?\n'+
        ' (s) by a starting number\n'+
        ' (f) from a file\n'+
        '?: ',default='s',allowed=['s','f'])
    if naming == 's':
        startnumber = int(myutils.ask_user('Input 6-digit starting number: '))
        numbering = None
    elif naming == 'f':
        startnumber = 0
        numbering = {}
        print 'Will use a column in %s to define h-numbering.' % inputfile
        filedata = []
        fh = open(inputfile,'r')
        for line in fh:
            line = line.strip()
            if not re.match('^#',line) and re.match('\w',line):
                cols = line.split()
                filedata.append([line,cols])
        fh.close()
        print 'First line reads...'
        print filedata[0][0]+'\n'
        hcolumn = int(myutils.ask_user('Input column number in file defining hnumber: '))
        choice = myutils.ask_user('How should hnumbers be defined using column %d?\n' % hcolumn+
                                  '1) first 6 digits in entry\n'+
                                  '2) first 6 digits in entry + offset\n'+
                                  '3) last 6 digits in entry\n'+
                                  '4) last 6 digits in entry + offset\n'+
                                  '?: ',allowed=('1','2','3','4'))
        if choice in ('2','4'):
            offset = int(myutils.ask_user('Input integer for numbering offset: '))
        elif choice in ('1','3'):
            offset = 0
        if choice in ('1','2'):
            for i in range(len(filedata)):
                tmp = filedata[i][1][hcolumn-1]
                name = ''
                for letter in tmp:
                    if letter.isdigit():
                        name += letter
                numbering[filedata[i][1][0]] = int(name) + offset
        elif choice in ('3','4'):
            for i in range(len(filedata)):
                tmp = filedata[i][1][hcolumn-1]
                name = ''
                for letter in tmp[::-1]:
                    if letter.isdigit():
                        name = letter + name
                numbering[filedata[i][1][0]] = int(name) + offset

outputfile = None
while not outputfile or os.path.exists(outputfile):
    outputfile = myutils.ask_user('Name of file to create: ')
    if os.path.exists(outputfile):
        overwrite = myutils.ask_user('%s already exists.  Overwrite it? (y/n) [n]: ' % outputfile,
                                     default='n',allowed=('y','n'))
        if overwrite == 'y':
            break

if pmyears != 0:
    dopm = True
else:
    dopm = False

rawdata = makeOlistFile1(inputfile,pmyears,dopm,startnumber,telescopeDEC=telescopeDEC,
                         hnumbersFromName=numbering,maxnumps=maxnumps,searchrad=searchrad)

if ourformat:
    outputfh = open(outputfile,'w')
    for i in range(len(rawdata)):
        outputfh.write(rawdata[i]['targ_line'])
        if includeComments and rawdata[i]['targ_comment']:
            outputfh.write(' #'+rawdata[i]['targ_comment'])
        outputfh.write('\n')
        if includePointSources:
            for j in range(len(rawdata[i]['ps_line'])):
                outputfh.write(rawdata[i]['ps_line'][j]+'\n')
    outputfh.close()
else:
    outputfh = open(outputfile,'w')
    for i in range(len(rawdata)):
        outputfh.write(rawdata[i]['targ_line2'])
        if includeComments and rawdata[i]['targ_comment']:
            outputfh.write(' #'+rawdata[i]['targ_comment'])
        outputfh.write('\n')
        if includePointSources:
            for j in range(len(rawdata[i]['ps_line2'])):
                outputfh.write(rawdata[i]['ps_line2'][j]+'\n')
    outputfh.close()

print 'Done'

Pick telescope (WIYN, GN or GS)? [WIYN]GS
Pick 'w' for wlist or 'g' for Gemini format [w]:w
-----------------------------------------------------------------
make_wlist is running with these settings:
Max number of point sources/science target = 1
Format = wlist format
Telescope = GS at Latitude = -30.2406
-----------------------------------------------------------------
Name of input data file [-f]:Howell.coords
Do you want a point source added for each target (y/n)? [y]y
Include trailing comments in the output file (y/n)? [n]y
How should H-numbers be assigned to targets?
 (s) by a starting number
 (f) from a file
?: s
Input 6-digit starting number: 100000
Name of file to create: Howell.wlist


Done


In [0]:
# To preview output file:

look = open(outputfile,'r')
print look.read()
look.close()

In [0]:
# Download wlist/gemini file to local computer:

files.download(outputfile)

# Files can also be accessed using '>' menu at top left of screen under 'Files' menu. Right click on desired file to delete or download.

##4) If observing at Gemini, append the Queue IDs to our target list using the CSV file returned by Gemini.

This script (copied from `attachQID.py`) will create a new wlist (*.wglist*) with the abbreviated Gemini Queue ID numbers appended to the end of each target line. You must upload the wlist and corresponding csv-formatted file returned from Gemini which associates the Queue IDs with the object names in the wlist. 



In [0]:
# Upload csv file from Gemini:

from google.colab import files
queueIDfile = files.upload()


Saving GS-2019A-Q-222.csv to GS-2019A-Q-222.csv


In [0]:
# Upload wlist file to be appended:

wlistfile = files.upload()

# **Currently you must upload a wlist rather than use one produced by make_wlist.py above**

Saving Howell.wlist to Howell (1).wlist


In [0]:
# Run attachQID.py:

import re
import argparse
from warnings import warn

"""
Append abbreviated Gemini Queue ID numbers to the end of each target
line in a wlist using a csv-formatted file returned from Gemini which
associates the Queue IDs with the object names in the wlist.
"""

def getTargetAssociations(fh):
    queuenumbers = {}
    for line in fh:
        if re.search('\w',line) and not line.startswith("#"):
            cols = [x.strip() for x in line.split(',')]
            if cols[1].startswith("HR"):
                name = cols[1].replace(' ','')
            else:
                name = cols[1].replace(' ','_')
            q = cols[0]
            queuenumbers[name] = q
    return queuenumbers

def readWlist(fh):
    lines = []
    objnames = []
    longestline = 0
    for line in fh:
        line = line.rstrip()
        objname = None
        if not line.startswith('#'):
            if '#' not in line:
                length = len(line)
            else:
                length = line.find('#') + 1
            if length>longestline:
                longestline = length
            cols = line.split()
            if len(cols)>=6 and re.match("H\d{6}",line):
                objname = (cols[5].split('/'))[1]
            elif line.startswith('HR'):
                objname = cols[0]+cols[1]
        lines.append(line)
        objnames.append(objname)
    return (lines,objnames,longestline)
        
#-------main program---------

q = queueIDfile.values()[0]
q = q.splitlines()

w = wlistfile.values()[0]
w = w.splitlines()

queue = getTargetAssociations(q)
wlist = readWlist(w)

qoutput = str(wlistfile.keys()[0])
qoutputfile = qoutput.rstrip('wlist')+'wglist'
qoutputfh = open(qoutputfile,'w')
          
     
missingids = []
for i in range(len(wlist[0])):
    line = wlist[0][i]
    objname = wlist[1][i]
    if objname in queue:
        mtch = re.match('(G(N|S)\-\d{4}(A|B)\-\w+\-)(.+)',queue[objname])
        if mtch:
            queue[objname] = mtch.group(4)
        comment = ' #'+queue[objname]
    elif not line.startswith('#'):
        comment = " # NO QID for this"
        missingids.append(objname)
    else:
        comment = ''

    if '#' in line and not line.startswith('#'):
        idx = line.find('#') 
        line1 = line[:idx]
        comment += ' '+line[idx+1:]
    else:
        line1 = line
        comment += ''
        
    fmt = '{:%ds} {:s}\n' % wlist[2]
    #    print fmt.format(line1,comment)
    #args.outputfile.write(fmt.format(line1,comment))
    qoutputfh.write(fmt.format(line1,comment))

qoutputfh.close()
if missingids:
    warn("Missing IDs for {} objects in {}".format(len(missingids), wlist.name))

print 'Done'

Done


In [0]:
# To look at output file:

look = open(qoutputfile,'r')
print look.read()
look.close()

In [0]:
# To save wglist in Google Drive:

with open(qoutputfile,'r') as temp:
  test = temp.read()

with open('/content/gdrive/My Drive/GS2019A/wlists/'+qoutputfile, 'w') as f:
   f.write(test)

## 5) Create nightly oberving plans by assembling individual targets from all programs being observed.



*   Start with a new text file, typically saved as a raw *olist* file named for the local date (e.g. *may20raw.olist*), and list relevant date/telescope/program information at top as desired.

*  Copy and paste individual science targets, and their associated PSF calibrators, from each *wlist* into the raw *olist*  in order of right ascension. Each target + calibrator should be bracketed by dashed lines. If more than one science target has the same calibrator they may be combined into one 'observation set' between dashed lines.

*  In general, try to select targets based on the priority (e.g. Band 1 vs. Band 3 at Gemini) and time allocated to each program, as well any priorities listed within programs. 

*  We also try to stick to small jumps in declination, selecting targets in clumps of declination and smoothly progressing back and forth between smaller and larger declinations (see example below).

* Depending on the brightness of each target, plan roughly 4 - 8 observation blocks per hour of RA. 

*  Add in calibration binaries, used to determine the plate scale, periodically throughout the night (usually in Howell program, listed as 'HOWELL_CALIB'). A known binary should also be observed with different rotation angles at least once per observing run.


Example:

```
05/20/19 EMCCD2 ZORRO
---------------------------
sunset = 17:49 = 8:47 LST
sunrise = 7:29 = 22:29 LST
-----------------------------------------------------------------
program ID       PI           hours(start)       hours(used)
---------------  -----------  ------------       -----------
GS-2019A-LP-101  Crossfield      20.0h
GS-2019A-Q-110   Mendez           8.5h
GS-2019A-Q-222   Howell          70.0h
GS-2019A-Q-230   Douglas         32.0h
GS-2019A-Q-302   Winters         20.0h 
GS-2019A-Q-311   Mendez           2.9h
-----------------------------------------------------------------
starting file no: S20190520Z0001
focus: -0.05
ROI: 256x256 (~2.5")
--------------------------------------------------------------------------------------------------------------
template:
H10000X 0001 03:00 1000 1000 10:10:10.0 +10:10:10 2000.00  10.00 PI/OBJECT #comment
--------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------
H610001        08:58:52.4 -43:27:10 2000.00   8.29  CROSSFIELD/30828562               #101-46
HR 3600        09:01:20.8 -41:51:52 2000.00                                           #101-68
--------------------------------------------------------------------------------------------------------------
H620108        09:10:04.4 -43:55:30 2000.00   9.07  HOWELL_TOIs/74482749              #222-33
HR 3672        09:14:08.2 -44:08:45 2000.00                                           #222-298
--------------------------------------------------------------------------------------------------------------
H620128        09:54:52.9 -23:19:56 2000.00  10.18  HOWELL_TOIs/101395259             #222-958
HR 3965        10:04:21.0 -24:17:08 2000.00                                           #222-229
--------------------------------------------------------------------------------------------------------------
HR 4189        10:43:01.8 +26:19:32 2000.00                                           #222-727
H620049        11:14:33.2 +25:42:37 2000.00   7.70  HOWELL_RODRIGUEZ/HD97658          #222-358
--------------------------------------------------------------------------------------------------------------
HR 4356        11:13:45.6 -00:04:11 2000.00                                           #222-469
H620036        11:16:28.1 -03:58:32 2000.00  15.60  HOWELL/EPIC201205469              #222-67 K2-43
--------------------------------------------------------------------------------------------------------------
H620015        11:20:51.8 -23:13:02 2000.00   8.00  HOWELL_KANE/HD98649               #222-919
HR 4428        11:29:38.6 -24:27:48 2000.00                                           #222-765
--------------------------------------------------------------------------------------------------------------
H660006        11:46:21.1 -27:57:49 2000.00   7.55  MENDEZ_110/HIP57421   #110-70
H660007        11:48:45.1 -26:44:59 2000.00   5.11  MENDEZ_110/HR4532     #110-67
```


##6) At the conclusion of each night of observing, a final *olist* must be produced based on the nightly observing plan and the log from the '*endnight*' script at the telescope.



*   Copy and paste the **file number**, **time of observation**, and **blue and red gains** listed in the '*endnight*' log (see specklegroup@gmail.com) from the first observation of each target into the observing schedule between the **H-number/HR number** and **right ascension** of the corresponding target. If you are just updating the target list, only one line per target is needed (one line per file is needed for data reduction).

* Duplicate the edited *olist* file, then delete all unobserved targets, ensuring that each remaining line begins with the H-number/HR number and has 10 columns.



Example:



```
--------------------------------------------------------------------------------------------------------------
H620015        11:20:51.8 -23:13:02 2000.00   8.00  HOWELL_KANE/HD98649               #222-919
HR 4428        11:29:38.6 -24:27:48 2000.00                                           #222-765
--------------------------------------------------------------------------------------------------------------
H660006 0400 01:25   10   10 11:46:21.1 -27:57:49 2000.00   7.55  MENDEZ_110/HIP57421   #110-70 seeing ~1"
H660007 0403 01:35   20    2 11:48:45.1 -26:44:59 2000.00   5.11  MENDEZ_110/HR4532     #110-67
--------------------------------------------------------------------------------------------------------------
H650002 0404 01:46  100  100 12:01:46.1 -34:39:01 2000.00   6.96  MENDEZ_311/HIP58669   #311-9 
H650003 0409 01:52  100  100 12:23:35.4 -35:24:46 2000.00   5.31  MENDEZ_311/HR4712     #311-11
--------------------------------------------------------------------------------------------------------------
H660008 0410 01:59   40   50 12:15:10.6 -10:18:45 2000.00   6.11  MENDEZ_110/HIP59750   #110-90 seeing ~1.2"
H660009 0413 02:02   40   50 12:20:55.7 -13:33:57 2000.00   5.14  MENDEZ_110/HR4699     #110-81


```







##7) After each night of observing the target lists (*wglists*) for each program must also be updated.

* On the **first night** of observing, add a line to the end of each *wglist* that says 'observed' as shown below. If there are targets that need to be observed more than once, such as plate scale binaries, bracket off those observations with '**#begin_permanent**' and '**#end_permanent**' so they are not marked as observed and removed from the list.

```
        Observed:----------------------------------------------------
```

* On the **first night**, also create a text file named `wlists4update.dat` containing the names and locations of all *wglists* to be updated (**in next cell**).

In [0]:
# Make wlists4update.dat file & write to Google Drive (if it doesn't already exist):

wglistfiles = ['Crossfield.wglist','Douglas.wglist','Howell_orig.wglist',
               'Mendez110.wglist','Mendez311.wglist','Winters.wglist']

wglistpath = '/content/gdrive/My Drive/GS2019A/wlists/'


with open('/content/gdrive/My Drive/GS2019A/wlists/wlists4update.dat', 'w') as f:
  for n in wglistfiles:
      f.write(wglistpath+n+'\n')


*  After **each night** run the script `updatewlists.py` (**in next cell**) to update all *wglist* files based on the night's final *olist* (which should list only observed targets). You need to pass the location of the *wlists4update.dat* file and the relevant *olist* file to the script as shown below. 

```
        updatewlists.py -l "/wlists/wlists4update.dat" "/olists/may20.olist"
```

*  New *wglist* files should be created with the date of observation appended to the end of the file name. Verify that the date of the corresponding *olist* has been added to the bottom of each file and targets that were observed are now listed below the date. Note that once a target has been observed, its' PSF calibrator is no longer listed. If the revised *wglists* check out, rename them as the official *wglists*.

In [0]:
# Run updatewlists.py (Needs -l "full path to wlists4update.py" and "full path to olist"):

%run "/content/gdrive/My Drive/Obs_Prep_Notebook/updatewlists.py" -l "/content/gdrive/My Drive/GS2019A/wlists/wlists4update.dat" '/content/gdrive/My Drive/GS2019A/olists/may20.olist' 

## 8) Create a new observing plan for the next night using the updated *wglist* files.

*  Open duplicate *raw olist* from the previous night and delete the lines corresponding to objects that were observed. Rename the *olist* as tonight's raw *olist*
*  Add objects and adjust the contents of tonight's *olist* file by cutting and pasting the lines from the *wglists*. 
* Verify there are no duplicates using: `python checkduplicates.py -o olists/oct29raw.olist`.
* Update notes at top of *olist* if needed.

## 9) Repeat steps 6 - 8 for each night of an observing run.